In [1]:
 import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

Using TensorFlow backend.


In [2]:
from google.colab import files
f = files.upload()

Saving corpus_data.xlsx to corpus_data.xlsx
Saving mar.txt to mar.txt
Saving Translation Dataset.xlsx to Translation Dataset.xlsx


In [6]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2555664543862717514
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 1138988878442132764
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 5419545093096554901
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15956099072
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16064770871497363144
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
]


In [2]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [0]:
df_1 = pd.read_excel('corpus_data.xlsx')

In [0]:
df_2 = pd.read_csv('mar.txt',sep = '\t')

In [0]:
df_2.columns = ['eng','mar','..']
df_2 = df_2[['eng','mar']]

In [0]:
df_1.columns =  df_2.columns

In [0]:
frame_combined = pd.concat([df_1, df_2], ignore_index=True)

In [0]:
lines = frame_combined.sample(50000)

In [8]:
len(lines)

50000

In [0]:
lines = lines.reset_index()

In [0]:
lines = lines[['eng','mar']]

In [0]:
lines.eng = lines.eng.astype('str')
lines.mar = lines.mar.astype('str')

In [12]:
lines.head(5)

,eng,mar
0,I'll go with Tom.,मी टॉमबरोबर जाईन.
1,How can you translate what you don't understand?,जे तुला समजत नाही त्याचा तू अनुवाद कसा करू शकतोस?
2,Will you give me some?,तू मला थोडंसं देशील का?
3,Call me before you leave.,जाण्यापूर्वी मला फोन कर.
4,You should exercise.,तु व्यायाम करायला पाहिजेस.


In [0]:
# Lowercase all characters
lines.eng=lines.eng.apply(lambda x: x.lower())
lines.mar=lines.mar.apply(lambda x: x.lower())

In [0]:
# Remove quotes
lines.eng=lines.eng.apply(lambda x: re.sub("'", '', x))
lines.mar=lines.mar.apply(lambda x: re.sub("'", '', x))

In [0]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.mar=lines.mar.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [0]:
# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines.eng=lines.eng.apply(lambda x: x.translate(remove_digits))
lines.mar = lines.mar.apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

In [0]:
# Remove extra spaces
lines.eng=lines.eng.apply(lambda x: x.strip())
lines.mar=lines.mar.apply(lambda x: x.strip())
lines.eng=lines.eng.apply(lambda x: re.sub(" +", " ", x))
lines.mar=lines.mar.apply(lambda x: re.sub(" +", " ", x))

In [0]:
# Add start and end tokens to target sequences
lines.mar = lines.mar.apply(lambda x : 'START_ '+ x + ' _END')

In [19]:

lines.sample(10)

,eng,mar
13539,i watch television,START_ मी टीव्ही बघतो _END
43165,no one knows tom is here,START_ टॉम इथे आहे हे कोणालाच ठाऊक नाहीये _END
25563,i put on my cap to go out,START_ मी बाहेर जायला माझी टोपी घातली _END
26654,he let me stay for a night,START_ त्याने मला एका रात्रीसाठी राहायला दिलं ...
14856,nothing can stop us now,START_ आता आपल्याला कोणीच थांबवू शकत नाही _END
32202,there was never this much snow before,START_ आधी इतका बर्फ कधीच नसायचा _END
44022,i dont want to take your money,START_ मला तुझे पैसे घ्यायचे नाहीयेत _END
29719,tom is going abroad next year,START_ टॉम पुढच्या वर्षी विदेशात जाणार आहे _END
14691,i was hungry and angry,START_ मी भूक लागली होती आणि राग आला होता _END
38449,hes watching me,START_ तो मला बघतोय _END


In [0]:
# Vocabulary of English
all_eng_words=set()
for eng in lines.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

# Vocabulary of French 
all_marathi_words=set()
for mar in lines.mar:
    for word in mar.split():
        if word not in all_marathi_words:
            all_marathi_words.add(word)

In [21]:
# Max Length of source sequence
lenght_list=[]
for l in lines.eng:
    lenght_list.append(len(l.split(' ')))
max_length_src = np.max(lenght_list)
max_length_src

61

In [22]:
# Max Length of target sequence
lenght_list=[]
for l in lines.mar:
    lenght_list.append(len(l.split(' ')))
max_length_tar = np.max(lenght_list)
max_length_tar

49

In [23]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_marathi_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_marathi_words)
num_encoder_tokens, num_decoder_tokens

(6979, 16048)

In [24]:
num_decoder_tokens += 1 # For zero padding
num_decoder_tokens

16049

In [0]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [0]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [27]:
lines = shuffle(lines)
lines.head(10)

,eng,mar
19145,what are you going to do,START_ तुम्ही काय करणार आहात _END
36329,youre a monster,START_ तू राक्षस आहेस _END
1649,the child was sleeping on its mothers lap,START_ मूल आपल्या आईच्या मांडीवर झोपलेलं _END
47029,i think its unlikely that the next version of ...,START_ विंडोजची पुढची आवृत्ती या महीन्याच्या श...
34115,im trying to learn french,START_ मी फ्रेंच शिकायचा प्रयत्न करतेय _END
39967,tom is standing nearby,START_ टॉम जवळपास उभा आहे _END
43466,i can ski,START_ मला स्की करता येतं _END
39119,she still loves him,START_ ती अजूनही त्याच्यावर प्रेम करते _END
30463,i heard it too,START_ ते मीसुद्धा ऐकलं _END
531,theres a sandwich here,START_ इथे एक सँडविच आहे _END


In [28]:
# Train - Test Split
X, y = lines.eng, lines.mar
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
X_train.shape, X_test.shape

((45000,), (5000,))

In [29]:
pwd

'/content'

#### Save the train and test dataframes for reproducing the results later, as they are shuffled.

In [0]:

X_train.to_pickle('/content/X_train.pkl')
X_test.to_pickle('/content/X_test.pkl')

In [0]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

### Encoder - Decoder Model Architecture

In [0]:
latent_dim = 50

In [33]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [35]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [0]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 256
epochs = 50

In [ ]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

### Always remember to save the weights

In [0]:
model.save_weights('nmt_weights.h5')

### Load the weights, if you close the application

In [0]:
model.load_weights('nmt_weights.h5')

### Inference Setup

In [0]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

### Decode sample sequeces

In [0]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

### Evaluation on Train Dataset

In [0]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [53]:
for k in range(0,100):
  (input_seq, actual_output), _ = next(train_gen)
  decoded_sentence = decode_sequence(input_seq)
  print('Input English sentence:', X_train[k:k+1].values[0])
  print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
  print('Predicted Marathi Translation:', decoded_sentence[:-4])
  print('=====================================================')

Input English sentence: its not easy to translate a poem in a foreign language
Actual Marathi Translation:  एखाद्या कवितेचा एका विदेशी भाषेत अनुवाद करणं सोपं नसतं 
Predicted Marathi Translation:  एका तास जर एका दिवसात नाही अनुवाद आहे हे मला सुरू
Input English sentence: tom will tell mary everything
Actual Marathi Translation:  टॉम मेरीला सर्वकाही सांगेल 
Predicted Marathi Translation:  टॉम मेरीला सर्वकाही सांगितलं 
Input English sentence: are you all right
Actual Marathi Translation:  तु बरा आहेस का 
Predicted Marathi Translation:  तू बरोबर आहेस का 
Input English sentence: what happened to your leg
Actual Marathi Translation:  तुझ्या पायाला काय झालं 
Predicted Marathi Translation:  काय झालं हे टॉमला हवं होतं 
Input English sentence: this is our room
Actual Marathi Translation:  ही आमची खोली आहे 
Predicted Marathi Translation:  ही आपली खोली आहे 
Input English sentence: its a strip club for women
Actual Marathi Translation:  हा स्त्रियांसाठी स्ट्रिप क्लब आहे 
Predicted Marathi Translatio

### Evaluation on Validation Dataset

In [0]:
val_gen = generate_batch(X_test, y_test, batch_size = 1)
k=-1

In [57]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: dinners ready
Actual Marathi Translation:  जेवण तयार आहे 
Predicted Marathi Translation:  तयार तर चालेल 


In [58]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: tom loves reading
Actual Marathi Translation:  टॉमला वाचनाची मोठी आवड आहे 
Predicted Marathi Translation:  टॉमला वाचत असताना 


In [59]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: what are you reading
Actual Marathi Translation:  तु काय वाचतेयस 
Predicted Marathi Translation:  तू काय वाचत आहेस 


### [The End]